In [35]:
#!pip install tensorflow==2.5.0
!pip install transformers

!pip install tensorflow_addons
import tensorflow_addons as tfa

In [36]:
PROJECT_PATH='/content/drive/MyDrive/OnlineCourses/MCI-NLP/Final-Prject/deeppavlov_resturants'
PROJECT_DATA='/content/drive/MyDrive/OnlineCourses/MCI-NLP/Final-Prject/deeppavlov_resturants/data'

In [37]:
import json
# with open(PROJECT_DATA+"/ner.json", 'r') as f:
#     NER_dict = json.load(f) 
    
# with open(PROJECT_DATA+"/intent.json", 'r') as f:
#     INTENT_dict = json.load(f) 

with open(PROJECT_DATA+"/ner_intent.json", 'r') as f:
    ner_intent = json.load(f) 

# counter = 128
# new_ner = {}
# for t in ner_intent:
#     new_ner[t] = ner_intent[t]
#     counter = counter-1
#     if counter<0:
#         break 
# ner_intent = new_ner

In [38]:
# NER_INTENT = {k: NER_dict[k] for k in NER_dict if k in INTENT_dict }

# INTENT_NER = {k: INTENT_dict[k] for k in INTENT_dict if k in NER_dict}
# print(len(NER_INTENT),' ? ',len(INTENT_NER))

NER_ITEMS = sorted( set(  [ t2 for t1 in ner_intent for t2 in ner_intent[t1]['NERTAGS']] ),reverse=True )  #'NETTAGS'
print(len(NER_ITEMS), NER_ITEMS)

INTENT_ITEMS = sorted( set(  [ t2 for t1 in ner_intent for t2 in ner_intent[t1]['INTENTS']] ),reverse=True )  #'NETTAGS'
print(len(INTENT_ITEMS),INTENT_ITEMS)

11 ['O', 'I-this', 'I-pricerange', 'I-name', 'I-food', 'I-area', 'B-this', 'B-pricerange', 'B-name', 'B-food', 'B-area']
27 ['unknown', 'thankyou', 'restart', 'request_pricerange', 'request_postcode', 'request_phone', 'request_food', 'request_area', 'request_addr', 'reqmore', 'reqalts', 'repeat', 'negate', 'inform_this', 'inform_pricerange', 'inform_name', 'inform_food', 'inform_area', 'hello', 'deny_name', 'deny_food', 'confirm_pricerange', 'confirm_food', 'confirm_area', 'bye', 'affirm', 'ack']


In [39]:
import tensorflow as tf
from transformers import AutoTokenizer

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [40]:
def encode_texts(tokenizer, texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="tf")

texts = [d for d in ner_intent]
tds = encode_texts(tokenizer, texts)
tds.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [41]:
encoded_texts = tds

In [42]:
intent_map = dict() # index -> intent
for idx, ui in enumerate(INTENT_ITEMS):
    intent_map[ui] = idx
intent_map

{'ack': 26,
 'affirm': 25,
 'bye': 24,
 'confirm_area': 23,
 'confirm_food': 22,
 'confirm_pricerange': 21,
 'deny_food': 20,
 'deny_name': 19,
 'hello': 18,
 'inform_area': 17,
 'inform_food': 16,
 'inform_name': 15,
 'inform_pricerange': 14,
 'inform_this': 13,
 'negate': 12,
 'repeat': 11,
 'reqalts': 10,
 'reqmore': 9,
 'request_addr': 8,
 'request_area': 7,
 'request_food': 6,
 'request_phone': 5,
 'request_postcode': 4,
 'request_pricerange': 3,
 'restart': 2,
 'thankyou': 1,
 'unknown': 0}

In [43]:
# map to train_data values
# def encode_intents(intents, intent_map):
#     encoded = []
#     for i in intents:
#         encoded.append(intent_map[i])
#     # convert to tf tensor
#     return tf.convert_to_tensor(encoded, dtype="int32")


import numpy as np
# # map to train_data values
def encode_intents(intents, intent_map):
    encoded = np.zeros((len(intents),len(intent_map)))
    for idx,intnt in enumerate(intents):
        encoded[ idx, [intent_map[i] for i in intnt]] = 1
    # convert to tf tensor

    return tf.convert_to_tensor(encoded, dtype="int32")

encoded_intents = encode_intents([ ner_intent[t]['INTENTS'] for t in ner_intent], intent_map)
encoded_intents

<tf.Tensor: shape=(1933, 27), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int32)>

In [44]:
np.argmax(encoded_intents[0]),ner_intent[next(iter(ner_intent))]

(14,
 {'INTENTS': ['inform_pricerange'],
  'NERTAGS': ['B-pricerange', 'O'],
  'NERVLAS': {'pricerange': 'cheap'},
  'TEXT': ['cheap', 'restaurant']})

In [45]:
slot_map = dict() # slot -> index
for idx, us in enumerate(NER_ITEMS):
    slot_map[us] = idx
slot_map

{'B-area': 10,
 'B-food': 9,
 'B-name': 8,
 'B-pricerange': 7,
 'B-this': 6,
 'I-area': 5,
 'I-food': 4,
 'I-name': 3,
 'I-pricerange': 2,
 'I-this': 1,
 'O': 0}

In [46]:
import numpy as np

# find the max encoded test length
# tokenizer pads all texts to same length anyway so
# just get the length of the first one's input_ids
#max_len = len(encoded_texts["input_ids"][0])
max_len = max([len(x) for x in encoded_texts["input_ids"]])

def encode_slots(all_slots, all_texts, 
                 toknizer, slot_map, max_len=max_len):
    encoded_slots = np.zeros(shape=(len(all_texts), max_len), dtype=np.int32)
    
    for idx, text in enumerate(all_texts):
        enc = [] # for this idx, to be added at the end to encoded_slots
        
        # slot names for this idx
        slot_names = all_slots[idx]
        
        # raw word tokens
        # not using bert for this block because bert uses
        # a wordpiece tokenizer which will make 
        # the slot label to word mapping
        # difficult
        raw_tokens = text.split()

        # words or slot_values associated with a certain
        # slot_name are contained in the values of the
        # dict slots_names
        # now this becomes a two way lookup
        # first we check if a word belongs to any
        # slot label or not and then we add the value from
        # slot map to encoded for that word
        for rt,rt_slot_name in zip(raw_tokens,slot_names):
            # use bert tokenizer
            # to get wordpiece tokens
            bert_tokens = tokenizer.tokenize(rt)
            
            # find the slot name for a token
            #rt_slot_name = get_slot_from_word(rt, slot_names)
            #rt_slot_name = get_slot_from_word(rt, slot_names) Moved to for and ziped
            if rt_slot_name is not None:
                # fill with the slot_map value for all ber tokens for rt
                enc.append(slot_map[rt_slot_name])
                enc.extend([slot_map[rt_slot_name]] * (len(bert_tokens) - 1))

            else:
                # rt is not associated with any slot name
                enc.append(0)
                
        
        # now add to encoded_slots
        # ignore the first and the last elements
        # in encoded text as they're special chars
        encoded_slots[idx, 1:len(enc)+1] = enc
    
    return encoded_slots
    

In [47]:
# all_slots = [td.slots for td in train_data]
# all_texts = [td.text for td in train_data]


all_slots = [ s['NERTAGS'] for t,s in ner_intent.items() ]
all_texts = [ t for t in ner_intent ]

In [48]:
encoded_slots = encode_slots(all_slots, all_texts, tokenizer, slot_map)
encoded_slots[2],all_slots[2],all_texts[2]
encoded_slots.shape

(1933, 22)

# **Classifier Model** 

In [49]:
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None,
                 model_name=model_name, dropout_prob=0.1):
        super().__init__(name="joint_intent_slot")
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        # self.intent_classifier = Dense(intent_num_labels,
        #                                name="intent_classifier")

        self.intent_classifier = Dense(intent_num_labels, activation='sigmoid',
                                       name="intent_classifier")

        self.slot_classifier = Dense(slot_num_labels,
                                     name="slot_classifier")

    def call(self, inputs, **kwargs):
        # two outputs from BERT
        trained_bert = self.bert(inputs, **kwargs)
        pooled_output = trained_bert.pooler_output
        sequence_output = trained_bert.last_hidden_state
        
        # sequence_output will be used for slot_filling / classification
        sequence_output = self.dropout(sequence_output,
                                       training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(sequence_output)

        # pooled_output for intent classification
        pooled_output = self.dropout(pooled_output,
                                     training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)
        return slot_logits, intent_logits
        #return {'slot': slot_logits,'intent': intent_logits} 

In [50]:
joint_model = JointIntentAndSlotFillingModel(
    intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [51]:
#https://towardsdatascience.com/multi-label-image-classification-in-tensorflow-2-0-7d4cf8a4bc72
@tf.function
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [52]:
# opt = Adam(learning_rate=3e-5, epsilon=1e-08)

# # two outputs, one for slots, another for intents
# # we have to fine tune for both
# losses = [SparseCategoricalCrossentropy(from_logits=True),
#           SparseCategoricalCrossentropy(from_logits=True)]

# metrics = [SparseCategoricalAccuracy("accuracy")]
# # compile model
# joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

In [53]:
encoded_slots.shape[1], encoded_intents.shape[1]

(22, 27)

In [54]:
categorcial_slots = tf.keras.utils.to_categorical(encoded_slots, num_classes=len(slot_map))
categorcial_slots.shape, categorcial_slots.shape[-1]

((1933, 22, 11), 11)

In [55]:
class MacroF1(tf.keras.metrics.Metric):

  def __init__(self, name='MacroF1', **kwargs):
    super(MacroF1, self).__init__(name=name, **kwargs)
    self.true_positives = self.add_weight(name='tp', initializer='zeros')

  def update_state(self, y_true, y_pred, sample_weight=None):
    thresh = 0.5
    y_hat =  tf.cast(tf.reshape(y_pred, [-1,11]),tf.float32)
   
    y2 = tf.cast(y_true, tf.float32)
    y2 = tf.keras.utils.to_categorical(y2, num_classes=11)
    y2 = tf.reshape(y2,[-1])
    #oh_labels = tf.cast(tf.one_hot(tf.cast(labels, tf.int32), num_classes), dtype=labels.dtype)
    #print(y)
    #y2 = tf.cast(tf.one_hot(tf.cast(y2, tf.int32), 11), dtype=tf.int32)
    y =  tf.cast(y2 , tf.float32)

    print(y.shape, y_hat)
    print(y.shape, y_hat)
    print(y.shape, y_hat)
    print(y.shape, y_hat)
    print(y.shape, y_hat)
    print(y.shape, y_hat)

    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)

  def result(self):
    return self.true_positives

In [56]:
tf.config.run_functions_eagerly(True)
class MY_F1Score(tfa.metrics.FBetaScore):
    def __init__(
        self,
        num_classes: 11,
        average = None,
        threshold = None,
        name = "MY_F1Score",
        dtype = None,
    ):
        super().__init__(num_classes, average, 1.0, threshold, name=name, dtype=dtype)


    def get_config(self):
        base_config = super().get_config()
        del base_config["beta"]
        return base_config

    def update_state(self, y_true, y_pred, sample_weight=None):   
        #print(y_true,y_pred) 
        #y_true_ = tf.keras.utils.to_categorical(y_true, num_classes=11)
        y_true_ = tf.constant(tf.reshape(y_true,[-1,y_true.shape[-1]])   )
        y_pred_ = tf.constant(tf.reshape(y_pred,[-1,y_pred.shape[-1]])   )
        #print(y_true_,y_pred_) 
 
        super().update_state(y_true_, y_pred_, sample_weight=None)

In [57]:
opt = Adam(learning_rate=3e-5, epsilon=1e-08)

# two outputs, one for slots, another for intents
# we have to fine tune for both
# losses = [SparseCategoricalCrossentropy(from_logits=True),
#           'binary_crossentropy'
#           #macro_soft_f1
#           ]

losses = [CategoricalCrossentropy(from_logits=True),
          'binary_crossentropy'
          #macro_soft_f1
          ]
                    
# losses = {'slot': CategoricalCrossentropy(from_logits=True),'intent': 'binary_crossentropy'}

#from tensorflow_addons import metrics
#metric1 = tfa.metrics.F1Score(num_classes=encoded_slots.shape[1], average = 'macro', threshold=0.7, name='m1')
metric2 = tfa.metrics.F1Score(num_classes=encoded_intents.shape[1], average = 'macro', threshold=0.7, name='m2')


#metrics = ['accuracy']
#metrics = [ metric1, metric2]
# metric1 = MY_F1Score(num_classes=11, average = 'micro', threshold=0.7, name='m1')
# metrics = {'slot':metric1, "'intent'":metric2} 
metrics = [ MY_F1Score(num_classes=11, average = 'micro', threshold=0.7, name='m1')]

# compile model
joint_model.compile(optimizer=opt, loss=losses, loss_weights = [1,3], metrics=metrics)

In [58]:
x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}

#history = joint_model.fit(x, (encoded_slots, encoded_intents), epochs=1, batch_size=32, shuffle=True)
history = joint_model.fit(x, (categorcial_slots, encoded_intents), epochs=6, batch_size=32, shuffle=True)
#history = joint_model.fit(x, {'slot': categorcial_slots,'intent': encoded_intents}, epochs=1, batch_size=32, shuffle=True)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/6
61/61 [==============================] - 35s 566ms/step - loss: 1.1489 - output_1_loss: 0.2468 - output_2_loss: 0.3007 - output_1_m1: 0.6716 - output_2_m1: 0.0422
Epoch 2/6
61/61 [==============================] - 34s 559ms/step - loss: 0.3524 - output_1_loss: 0.0356 - output_2_loss: 0.1056 - output_1_m1: 0.7935 - output_2_m1: 0.5108
Epoch 3/6
61/61 [==============================] - 34s 564ms/step - loss: 0.2274 - output_1_loss: 0.0194 - output_2_loss: 0.0693 - output_1_m1: 0.7955 - output_2_m1: 0.7562
Epoch 4/6
61/61 [==============================] - 34s 564ms/step - loss: 0.1655 - output_1_loss: 0.0113 - output_2_loss: 0.0514 - output_1_m1: 0.8006 - output_2_m1: 0.8297
Epoch 5/6
61/61 [==============================] - 34s 554ms/step - loss: 0.1238 - output_1_loss: 0.0077 - output_2_loss: 0.0387 - output_1_m1: 0.8087 - output_2_m1: 0.8768
Epoch 6/6
61/61 [==============================] - 35s 566ms/step - loss: 0.0979 - output_1_loss: 0.0057 - output_2_loss: 0.0307 - outp

In [ ]:
res = joint_model.predict(x)
res

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [ ]:
# test = MY_F1Score(num_classes=11, average = 'macro', threshold=0.7, name='m1')

# test.update_state(encoded_slots,res[0])
# test.result()

In [61]:
import numpy as np

a = np.random.random((10))
print(a,np.nonzero(a>0.5)[0])

[0.81436776 0.39760391 0.28431828 0.9049492  0.50967494 0.3507657
 0.51212164 0.60843194 0.19609155 0.40180292] [0 3 4 6 7]


In [62]:
def map2cat(t):
    return t.replace("B-", "").replace("I-", "") 

def nlu(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    #print(outputs)
    slot_logits, intent_logits = outputs

    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, :]
    #intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    intent_ids = np.nonzero(intent_logits.numpy()>0.8)[1]
    #print(intent_logits,np.nonzero(intent_logits.numpy()>0.5))

    #info = {"intent": intent_names[intent_id], "slots": {}}
    info = {"intent": [intent_names[intent_id] for intent_id in intent_ids], "slots": {}}
    
    out_dict = {}
    # get all slot names and add to out_dict as keys
    predicted_slots = set([slot_names[s] for s in slot_ids if s != 0])

    #merge slots
    predicted_slots = set([ map2cat(t) for t in predicted_slots])
    #print(predicted_slots)
    for ps in predicted_slots:
      out_dict[ps] = []

    # check if the text starts with a small letter
    if text[0].islower():
      tokens = tokenizer.tokenize(text, add_special_tokens=True)
    else:
      tokens = tokenizer.tokenize(text)
    for token, slot_id in zip(tokens, slot_ids):
        # add all to out_dict
        #slot_name = slot_names[slot_id]
        slot_name = map2cat(slot_names[slot_id])

        if slot_name == "O":
            continue

        # collect tokens
        collected_tokens = [token]
        idx = tokens.index(token)

        # see if it starts with ##
        # then it belongs to the previous token
        if token.startswith("##"):
          # check if the token already exists or not
          if tokens[idx - 1] not in out_dict[slot_name]:
            collected_tokens.insert(0, tokens[idx - 1])

        # add collected tokens to slots
        out_dict[slot_name].extend(collected_tokens)

    # process out_dict
    for slot_name in out_dict:
        tokens = out_dict[slot_name]
        slot_value = tokenizer.convert_tokens_to_string(tokens)

        info["slots"][slot_name] = slot_value.strip()

    return info

t_itrator=iter(ner_intent)
for i in range(3):
    tkey = next(t_itrator)
    print(tkey, nlu(tkey, tokenizer, joint_model, INTENT_ITEMS, NER_ITEMS), ner_intent[tkey]['INTENTS'], ner_intent[tkey]['NERVLAS'])
    print()

cheap restaurant {'intent': ['inform_pricerange'], 'slots': {'pricerange': 'cheap'}} ['inform_pricerange'] {'pricerange': 'cheap'}

any {'intent': [], 'slots': {}} ['inform_this'] {}

south {'intent': ['inform_area'], 'slots': {'area': 'south'}} ['inform_area'] {'area': 'south'}



In [63]:
nlu("i am looking for a fast food", tokenizer, joint_model, 
    INTENT_ITEMS, NER_ITEMS)

{'intent': ['inform_food'], 'slots': {'food': 'fast'}}

# State tracker

In [64]:
#load templates
def load_templates():
    templates = {}
    with open(PROJECT_DATA+"/simple-dstc2-templates.txt", 'r') as f:
        Lines = f.readlines()
    for line in Lines:
        key, val = line.split("\t")
        templates[key] = val.strip()
    
    return templates

templates = load_templates()
#templates

In [65]:
act_map = dict() # slot -> index
for idx, us in enumerate(templates.keys()):
    act_map[us] = idx
act_map

{'api_call': 0,
 'bye': 1,
 'canthear': 2,
 'canthelp_area': 3,
 'canthelp_area_food': 4,
 'canthelp_area_food_pricerange': 5,
 'canthelp_area_pricerange': 6,
 'canthelp_food': 7,
 'canthelp_food_pricerange': 8,
 'confirm-domain': 9,
 'expl-conf_area': 10,
 'expl-conf_food': 11,
 'expl-conf_pricerange': 12,
 'impl-conf_area+impl-conf_pricerange+request_food': 13,
 'impl-conf_food+impl-conf_pricerange+request_area': 14,
 'impl-conf_food+request_area': 15,
 'inform_addr+inform_food+offer_name': 16,
 'inform_addr+inform_phone+inform_pricerange+offer_name': 17,
 'inform_addr+inform_phone+offer_name': 18,
 'inform_addr+inform_postcode+offer_name': 19,
 'inform_addr+inform_pricerange+offer_name': 20,
 'inform_addr+offer_name': 21,
 'inform_area+inform_food+inform_pricerange+offer_name': 22,
 'inform_area+inform_food+offer_name': 23,
 'inform_area+inform_phone+offer_name': 24,
 'inform_area+inform_postcode+offer_name': 25,
 'inform_area+inform_pricerange+offer_name': 26,
 'inform_area+offer_n

In [66]:
import json 

import json
with open(PROJECT_DATA+"/slot_vals.json", 'r') as f:
    slot_vals = json.load(f)

def get_real_val(k,v):
    search_domain = slot_vals[k]
    for t in search_domain:
        if v in search_domain[t]:
            return t
    return "'"+v        
    


In [67]:
import json
with open(PROJECT_DATA+"/restaurants.json", 'r') as f:
    restaurants = json.load(f)
#print(restaurants)
def api_call (input_slots):
    print('search' , input_slots)
    res = []
    for restaurant in restaurants:
        #print( sear input_slots)
        for k, v in input_slots.items():
            if v== 'dontcare' or v==None:
                continue
            if k in restaurant and restaurant[k] != v :
                break
        else:
            res.append(restaurant)
    return res

In [70]:
import random



class BOT:

    unsuported_acts={'unknown',
                        'thankyou',
                        #'restart',
                        'reqmore',
                        #'reqalts',
                        'repeat',
                        'negate',
                        #'hello',
                        'deny_name',
                        'deny_food',
                        'confirm_pricerange',
                        'confirm_food',
                        'confirm_area',
                        #'bye',
                        'affirm',
                        'ack'}

    state0 = {  'prev_acts':[],
                'slots':{   'food':None,
                            'area': None,
                            'pricerange': None
                        },
                'recomanded':[],
                'restaurant_curser':None,
                'this':None,
                'repvious_this':False}
            
    def __init__(self, nul, joint_model,tokenizer,INTENT_ITEMS, NER_ITEMS, templates):
        self.nlu = nul
        self.joint_model = joint_model
        self.INTENT_ITEMS = INTENT_ITEMS
        self.NER_ITEMS = NER_ITEMS
        self.templates = templates
        self.tokenizer = tokenizer
        self.state = self.state0
   
        #print(self.render_template('welcomemsg'))

    def render_template(self, key,slots=[]):
        str_ = self.templates[key]
        #print("will return: ",str_)
        #for k,v in slots.items():
        for k,v in self.state['slots'].items():
            if v is not None:
                str_ = str_.replace('#'+k,v)
                #print('randering ', str_, ' ', '#'+k,' ' ,v)
        if self.state['restaurant_curser'] is not None:        
            print(self.state['restaurant_curser'])
            for k,v in self.state['restaurant_curser'].items():
                str_ = str_.replace('#'+k,v)
                    #print('randering ', str_, ' ', '#'+k,' ' ,v)       
        return str_  

    def match_(self, tokens):
        for k in self.templates:
            items = k.split('+')
            #print(items, tokens)
            if len(tokens) == len(items) and all(x in items for x in tokens):
                return k
             


    def next_act(self,intents, slots):
        intents = [item for item in intents if item not in self.unsuported_acts]
        if len(self.state['prev_acts']) ==0 and len(intents) == 0:
            return 'welcomemsg'
        #print(len(self.state['prev_acts']), len(intents))

        recom_list = api_call(self.state['slots'])
        #current = [x in recom_list not in x.name not in self.state[recomanded] ]
        if len(self.state['recomanded'])>0:
            # print(recom_list)
            # print(self.state['recomanded'])
            current = [x for x in recom_list if x['name'] not in [x1 for x1 in self.state['recomanded']] ]
        else:
            current = recom_list

        #print("\n\n\n\ncurrent",recom_list)
        #food #area #pricerange 'courntKnwledge' == '000'
        ck = '000'
        if self.state['slots']['food'] != None:
            ck = '100'
        if self.state['slots']['area'] != None:
            ck = ck[:1]+'10'
        if self.state['slots']['pricerange'] != None:
            ck = ck[:2]+'1'
        
        #print('ck', ck)
        print('address request checking', intents)

        requests = ['request_pricerange', 'request_postcode', 'request_phone',
                                'request_food', 'request_area', 'request_addr']
        if set(intents) <= set(requests):  
            tokens = [ t.replace('request','inform') for t in intents]
            print(tokens,self.match_(tokens+['inform_name']))
            #if len(current)>0:
            return self.match_(tokens+['offer_name'])
            #else:    
        else:
            print('address request didnt catch', intents)

        informs = [ 'inform_this', 'inform_pricerange', 'inform_name', 'inform_food', 'inform_area','reqalts']
        if any(x in informs for x in intents):
            print('he informes us let back sth or ask more ', ck)
            
            dual_fact=['110', '101', '101']
            if ck in dual_fact:
                if len(current)>0:
                    print('current[0].name',current[0].keys())
                    self.state['recomanded'].append(current[0]['name'])
                    self.state['restaurant_curser'] = current[0]
                    if ck == '110':
                        return 'inform_addr+inform_food+offer_name' 
                    if ck == '101':
                        return 'inform_food+inform_pricerange+offer_name' 
                    if ck == '011':
                        return 'inform_area+inform_pricerange+offer_name' 
                    # if ck=='111':
                    #     return 'inform_area+inform_food+inform_pricerange+offer_name'  
                else:        
                    if ck == '110':
                        return 'canthelp_area_food' 
                    if ck == '101':
                        return 'canthelp_food_pricerange' 
                    if ck == '011':
                        return 'canthelp_area_pricerange' 
                    # if ck=='111':
                    #     return 'canthelp_area_food_pricerange' 
            if ck=='111':
                if len(current)>0:
                    self.state['restaurant_curser'] = current[0]
                    self.state['recomanded'].append(current[0]['name'])
                    return 'inform_area+inform_food+inform_pricerange+offer_name'  
                else:
                    return 'canthelp_area_food_pricerange'

            #if len(intents)==1:
            #if intents[0] =='inform_food':
            if 'inform_food' in intents:
                if ck == '100':
                    if len(current) == 0:
                        return 'canthelp_food'
                    else:
                        return random.choice(['request_area','request_pricerange','impl-conf_food+request_area','impl-conf_food+request_area'])                        
            #if len(intents)==1 and 'inform_area':
            if 'inform_area' in intents:
                if ck == '010':
                    if len(current) == 0:
                        return 'canthelp_area'
                    else:
                        return random.choice(['request_food','request_pricerange','impl-conf_food+request_area','impl-conf_food+request_area'])                        

            #if len(intents)==1 and 'inform_pricerange':
            if 'inform_pricerange' in intents:
                if ck == '001':
                    if len(current) == 0:
                        return 'select_pricerange'
                    else:
                        return random.choice(['request_food','request_area'])
        # if 'reqalts' in intents:
        #     if len(current) == 0:




        #print('we are in last lines')
        if 'bye' in intents:
            return 'bye'
        
        if len(intents)==1 and (intents[0]=='hello' or  intents[0]=='restart'):
            self.state = self.state0
            return 'request_food'

        return  'confirm-domain'


    def __call__(self, talklist):
        conversation = []
        for talk in talklist:
            t = self.nlu(talk, tokenizer,joint_model,INTENT_ITEMS, NER_ITEMS)
            print("nlu: ",talk, t)
            for k,v in t['slots'].items():

                #self.state['slots'][k]=v
                self.state['slots'][k] = get_real_val(k,v)

            if 'inform_this' in t['intent'] and 'this' in t['slots']:
                #self.state['slots'][ self.state['this'] ] = t['slots']['this']
                self.state['slots'][ self.state['this'] ] = get_real_val(self.state['this'],t['slots']['this'])

            act_name = self.next_act(t['intent'],t['slots'])

            #print('the act in call',act_name)
            if 'request_food' == act_name:
                self.state['this'] = 'food'
            elif 'request_area' == act_name:
                self.state['this'] = 'area'
            elif 'request_pricerange' == act_name:
                self.state['this'] = 'pricerange'
            else:    
                self.state['this'] = None

            self.state['prev_acts'].append(act_name)
            response = self.render_template(act_name,self.state['slots'])
            #return self.render_template(act_name,self.state['slots'] )
            print(self.state)

            conversation.append('user: '+ talk)
            conversation.append('bot: '+  response)
            conversation.append('')

            # print('user: ')
            # print(talk)
            # print(' bot: ')
            # print(response)
        return conversation

#1-food 2-area 3-pricerange
bot = BOT(nlu, joint_model,tokenizer,INTENT_ITEMS, NER_ITEMS, templates)

# print (nlu("hi, i want to eat, can you suggest a place to go?", tokenizer, joint_model, 
#     INTENT_ITEMS, NER_ITEMS))
# bot = print(bot(['hi, i want to eat, can you suggest a place to go?']))


t = [
     'hi',
    #  'i am looking for chinese food',
    #  'what can i eat in city center',
    #  'where can i eat some thing chaep'
     'a cheap restaurant plz',
     'chinese',
     'address' ,  
     'how about west is there any there',
      ]
# print (nlu(t[-1], tokenizer, joint_model, 
#     INTENT_ITEMS, NER_ITEMS))
get_result = bot(t)

for t1 in get_result:
    print(t1)


nlu:  hi {'intent': [], 'slots': {}}
{'prev_acts': ['welcomemsg'], 'slots': {'food': None, 'area': None, 'pricerange': None}, 'recomanded': [], 'restaurant_curser': None, 'this': None, 'repvious_this': False}
nlu:  a cheap restaurant plz {'intent': ['inform_pricerange'], 'slots': {'pricerange': 'cheap'}}
search {'food': None, 'area': None, 'pricerange': 'cheap'}
address request checking ['inform_pricerange']
address request didnt catch ['inform_pricerange']
he informes us let back sth or ask more  001
{'prev_acts': ['welcomemsg', 'request_food'], 'slots': {'food': None, 'area': None, 'pricerange': 'cheap'}, 'recomanded': [], 'restaurant_curser': None, 'this': 'food', 'repvious_this': False}
nlu:  chinese {'intent': ['inform_food'], 'slots': {'food': 'chinese'}}
search {'food': 'chinese', 'area': None, 'pricerange': 'cheap'}
address request checking ['inform_food']
address request didnt catch ['inform_food']
he informes us let back sth or ask more  101
current[0].name dict_keys(['food',